# Programming Assignment 1: Search Algorithms
## Find a problem and develop a search algorithm to solve it. 

The problem or objective is to detect changes in micro-services and trigger pipelines for all impacted dependent services. We can represent the micro-services as a tree. Our DFS will leverage recursion and perform our DFS:
1. To locate the impacted node from the root of the tree
2. To locate the parent and sibling nodes of the impacted node.
3. To return a subtree of impacted nodes.

We recursively set an objective to build out `sets` and new trees. So there are multiple objectives being executed to verbosly illustrate the efficiency and process of DFS.

We use search algorithms to solve our problem and find the dependent services that have changed and trigger pipelines for them. The algorithm we are going to be implementing in our agent program is going to power a simple reflex agent. The acutators will be triggered by the sensors, which will detect changes in the services. Once a change is detected, the agent will trigger pipelines for all impacted dependent services. Our performance measure is to minimize the number of pipelines triggered. In a worst case scenario, we would trigger all pipelines for all services. Our best case scenario would be to trigger only the pipelines for the service that has changed. After evaluating search algorithms, we have decided to use a simple reflex agent with a depth-first search algorithm to find the impacted nodes. The environment is known, static, continuous, episodic and deterministic.

### Define the tree as a nested dictionary in order to perform our **Depth First Search**
I am choosing a dictionary, as I am most familiar with handling those. Other types, such as a list of lists could be used as well, or tupels. 

#### Ex. Data Structure 1: `generic_tree`

In [720]:
"""Goal Node: Target node to find in tree
Nodes: Tree structure with nested dictionaries representing nodes
For example:
{
    'A': {
        'B': {
            'C': {
                'D': {},
                'E': {}
            }
        }
    }
}

DFS Implementation:
1. find_subtree_actuator(tree, target_node):
   - Iteratively searches for target node using stack
   - Returns subtree if found, None if not found

2. get_all_descendants(subtree): 
   - Recursively collects all nodes in subtree
   - Returns set of descendant nodes

3. get_impacted_nodes_actuator(tree, target_node):
   - Entry point that combines other functions
   - Returns set of impacted nodes including target and descendants

Output: Target node + all descendant nodes in subtree
For example:
Input target 'B' -> Output {'B', 'C', 'D', 'E'}

Explanation: The DFS algorithm first locates the target node by traversing down 
the tree, then collects all descendants in its subtree to identify impacted nodes."""


print("Defining generic_tree tree...")
generic_tree = {
    "A": {
        "B": {
            "C": {
                "D": {
                    "E": {}  
                },
                "F": {
                    "G": {},    
                    "H": {}   
                },
                "I": {
                    "J": {}       
                }
            }
        }
    }
}
print("generic_tree:", generic_tree)

Defining generic_tree tree...
generic_tree: {'A': {'B': {'C': {'D': {'E': {}}, 'F': {'G': {}, 'H': {}}, 'I': {'J': {}}}}}}


`generic_tree`'s edges and nodes look like this.
```mermaid
graph TD;
    A --> B;
    B --> C;
    C --> D;
    D --> E;
    C --> F;
    F --> G;
    F --> H;
    C --> I;
    I --> J;
```

#### Ex. Data Structure 2: `descriptive_tree`

This data structure assigned to the `descriptive_tree` variable in the following cell is a nested dictionary, where each key is a service and the value is a dictionary of dependent services. It more accurately represents the architecture of a micro-services system. But is still just a tree.

In [721]:
print("Defining descriptive_tree...")
descriptive_tree = {
    "Frontend UI": {
        "Auth Service": {
            "API Gateway": {
                "User Service": {
                    "Profile Service": {}  
                },
                "Order Service": {
                    "Payment Service": {},    
                    "Inventory Service": {}   
                },
                "Notification Service": {
                    "Email Service": {}       
                }
            }
        }
    }
}
print("descriptive_tree:", descriptive_tree)

Defining descriptive_tree...
descriptive_tree: {'Frontend UI': {'Auth Service': {'API Gateway': {'User Service': {'Profile Service': {}}, 'Order Service': {'Payment Service': {}, 'Inventory Service': {}}, 'Notification Service': {'Email Service': {}}}}}}


`descriptive_tree`'s edges and nodes look like this.
```mermaid
flowchart TD
    A["Frontend UI"] --> B["Auth Service"]
    B --> C["API Gateway"]
    C --> D["User Service"]
    D --> E["Profile Service"]
    C --> F["Order Service"]
    F --> G["Payment Service"]
    F --> H["Inventory Service"]
    C --> I["Notification Service"]
    I --> J["Email Service"]
```

We can see in the both `generic_tree` and `descriptive_tree` are the same. `generic_tree` is letters. `descriptive_tree` is labeled more like our Task Environment.

### State
Now we need to implement a sensor in our task environment. A real distributed system is much more complex. For the sake of our implementation we will assume that an actuator exists that will be sending a precept to this sensor we are implementing. We initialize our `current_precet` as `None`. The Agent Program then randomly generates mock precepts, so that we can trigger an actuator based upon this value. When the program runs for the first time it will never have gotten a precept sent to it. 


```python
def sensor(node: str = None, last_precept: str = None, current_precept: str = None)
```

### Precept
Our precept for this portion of the Agent Program we're implementing here will be a "git sha."

They look like this: `9925fb3505cf60814a61a80cddb5a6f581257d49` and it's just of type string. 

Changes in these strings indicate that changes have occured in the code repository. In a CI/CD Environment, it is common to observe for changes Especially in code repositories. This AI we are writing will injest this precept and check if it is different that the previous one it has stored.
```mermaid
flowchart LR
    A["Local Commit"] --> B["Push to Remote"]
    B --> C["Emit Commit SHA"]
    C --> E["AI Agent"]
    E --> D["CI Pipeline Trigger"]

    subgraph E["AI Agent"]
        direction LR
        X["mock_data_stream_emitter()"] --> Y["sensor()"]
    end
```

## Implementing DFS as an Agent Function Between 2 Sensors Exchanging Precepts

From here on we will depart from the domain language regarding to an AI for CI/CD and stick by using the `generic_tree`. That way we can focus on what is happening in our implementation rather than the task environment.

We need something to mock the other parts of our program, because no other actuators exist. As stated above, we will have to write a bit of code to fake it for our program to run. After we define our program, we will demonstrate all of the functionality by running the `sensor()` function at the very end.


In [727]:
# Common utility function to get a random node from a tree to be used in the DFS Algorithm
# Instead of manually changing the node to be searched, we can use this utility function to get a random node from the tree
import random

def get_random_node(tree):

    # we will store all the nodes in the tree in this list
    all_nodes = []

    # Recursive function to traverse the tree and store all the nodes in the list
    def traverse(subtree):
        # iterate over the children of the current node
        for node, children in subtree.items():
            # add the current node to the list
            all_nodes.append(node)
            # recursively traverse the children
            traverse(children)

    # start the traversal from the root of the tree
    traverse(tree)

    # return a random node from the list of all nodes  
    return random.choice(all_nodes)

# This way we will have a random node to pass as the first argument each time we run sensor()
# Execute the cell to see the random node selected
print(f" • get_random_node • Random node selected: {get_random_node(generic_tree)}")

# For our example we are going to randomly change the precept value
def mock_data_stream_emitter(precept):
    # We are going to change the precept value 50% of the time
    if random.random() < 0.5:
        # when we change the precept value we are going to return a random SHA value by generating a random string of 40 characters with the characters 'abcdef1234567890'
        return ''.join(random.choices('abcdef1234567890', k=40))
    # The other 50% of the time we are going to return the current precept value
    return precept

 • get_random_node • Random node selected: C


In [723]:
#  DFS Implementation broken in to three functions
# 1. find_subtree: in regards to DFS this function finds the subtree of the target node by traversing the tree
# 2. get_all_descendants : in regards to DFS this function gets all the descendants of the subtree by recursively traversing the tree
# 3. get_impacted_nodes: in regards to DFS this function gets all the impacted nodes by calling the find_subtree and get_all_descendants functions

def find_subtree_actuator(tree: dict, target_node: str, key: str = None):
    # This function takes in `tree`, `target_node` and `key` as arguments. 
    # `tree` is the tree we are going to search on
    # `target_node` is the node we are looking for
    # `key` is the key of the tree we are currently on

    # It is the core of the Depth First Search Algorithm
    
    # We are going to use a stack and a while loop to traverse the tree to traverse the tree
    stack = [(tree, None)]
    while stack:
        # We are going to pop the last element of the stack and assign it to current and `_`
        # We are using `_` here to keep track of the parent node. It is assigned to None initially, but ends up being the parent node of the current node
        current, _ = stack.pop()
        
        # If the key is not None and the key is equal to the target node we are going to return the current subtree
        if target_node in current:
            # We have found the target node
            return current[target_node]

        # We are going to iterate over the current subtree in order to find the target node
        for key, subtree in current.items():
            stack.append((subtree, key))
    
        
    
    # If the target node is not found we are going to return None
    print(f" • find_subtree_actuator • Subtree for target '{target_node}': {subtree}")
    # If the target node is not found we are going to return None
    return None

# Once we have the subtree we are going to get all the descendants of the subtree
def get_all_descendants(subtree):
    # We do this by using a set to keep track of the nodes
    nodes = set()
    # We are going to use a recursive function to traverse
    for key, children in subtree.items():
        # We are going to add the key to the nodes set
        nodes.add(key)
        # We are going to call the get_all_descendants function recursively to get all the descendants of the children
        nodes.update(get_all_descendants(children))
    # We are going to return the nodes set   
    print(f" • get_all_descendants • Descendants: {nodes}") 
    return nodes

def get_impacted_nodes_actuator(tree: dict, target_node: str):
    # This log confirms the tree and target_node values we want to search on
    print(f" • get_impacted_nodes_actuator •these are the values of the tree and target_node {tree} and {target_node}")
    
    # We are going to call the find_subtree_actuator function to get the subtree of the target node
    subtree = find_subtree_actuator(tree, target_node)
    
    # Let's show what we found
    print(f" • get_impacted_nodes_actuator • subtree: {subtree}")
    
    # If the subtree is None we are going to return an empty set as there are no impacted nodes
    if subtree is None:
        return set()

    # We are going to call the get_all_descendants function to get all the descendants of the subtree
    impacted = {target_node}
    impacted.update(get_all_descendants(subtree))
    
    print(f" • get_impacted_nodes_actuator • Impacted nodes: {impacted}")

    # Here we are going to set the global variable impacted_nodes to the impacted nodes so it can be accessed outside of the function and logged.
    global impacted_nodes
    impacted_nodes = impacted
    return impacted


In [724]:

# Implement the sensor function
def sensor(node: str = None, last_precept: str = None, current_precept: str = None): 

    # Here we emulate a data stream that our sensor function would receive
    # For examle's sake we emulate a stream of 42 data points defined by the range function
    for i in range(42):
        # We ant to ouput the current iteration of the loop to illustrate the point and how it's evaluating
        
        print(f"<----------<----------<----------<----------<----------] Begin Sensor Iteration {i+1}  [-------------->---------->---------->")

        mock_data_stream_emitter(current_precept)
        
        # This shows the current precept value
        print(f" • sensor • current_precept: {current_precept}")
        
        # Here we handle what happens on initialisation of the sensor function
        if current_precept is None:
            # If the current precept value is None we are going to set it to a random SHA value
            current_precept = mock_data_stream_emitter(current_precept)
        else:
            # Otherwise we are going to set it to a random SHA value 50% of the time
            current_precept = mock_data_stream_emitter(current_precept)
        
        # Log the current precept value
        print(f" • current_precept: {current_precept}")
        
        # Here we are going to check if the current precept value is different from the last precept value. If it is we are going to print out the change and the node that it changed for
        if current_precept != last_precept:
            # We are going to set the changed_precept variable to the current precept value
            changed_precept = current_precept

            # Log the change
            print(f" • sensor • changed_precept: {changed_precept}")
            print(f" • sensor • string changed for Node: {node}, {last_precept}: is now {current_precept}")
            print(f" • sensor • last_precept: {last_precept}")

            # Set the last precept value to the current precept value
            last_precept = current_precept
            print(f" • sensor • current_precept: {current_precept}")
            
            # We are also going to call the get_impacted_nodes_actuator function to get the impacted nodes with our DFS algorithm
            # After we handle updating the precept above, we are going to call the get_impacted_nodes_actuator function to get the impacted nodes using DFS
            # We are going to pass the generic_tree and the node that we are currently on to the function
            get_impacted_nodes_actuator(generic_tree, node)
            
            print(f" • sensor • Triggering pipelines for {impacted_nodes} to discover impact from changes detected in '{changed_precept}'")
        else:
            print(" • sensor • No change detected.")
        print(f"<----------<----------<----------<----------<----------] End Sensor Iteration {i+1} [----------------->---------->---------->")
        



In [725]:
# Execute this cell to bring it all together. It will emulate the sensor function and show the output of the DFS algorithm.
# It will show various outcomes of the sensor function and the DFS algorithm, under different conditions.
sensor(get_random_node(generic_tree))


<----------<----------<----------<----------<----------] Begin Sensor Iteration 1  [-------------->---------->---------->
 • sensor • current_precept: None
 • current_precept: None
 • sensor • No change detected.
<----------<----------<----------<----------<----------] End Sensor Iteration 1 [----------------->---------->---------->
<----------<----------<----------<----------<----------] Begin Sensor Iteration 2  [-------------->---------->---------->
 • sensor • current_precept: None
 • current_precept: 91cca847e85b19cb79d247467033cf0d34aafc22
 • sensor • changed_precept: 91cca847e85b19cb79d247467033cf0d34aafc22
 • sensor • string changed for Node: H, None: is now 91cca847e85b19cb79d247467033cf0d34aafc22
 • sensor • last_precept: None
 • sensor • current_precept: 91cca847e85b19cb79d247467033cf0d34aafc22
 • get_impacted_nodes_actuator •these are the values of the tree and target_node {'A': {'B': {'C': {'D': {'E': {}}, 'F': {'G': {}, 'H': {}}, 'I': {'J': {}}}}}} and H
 • get_impacted_n

## Example Outputs




### When there is No change (There is no cost.)
 - `• sensor • current_precept: None`
 - `• current_precept: None`
 - `• sensor • No change detected`


```mermaid
graph TD;
    A --> B;
    B --> C;
    C --> D;
    D --> E;
    C --> F;
    F --> G;
    F --> H;
    C --> I;
    I --> J;

    style A fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style B fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style C fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style D fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style E fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style F fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style G fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style H fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style I fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style J fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
```

### Change in G (Requires DFS for the node, parent, and siblings. Done recursively.)
 - `• sensor • current_precept: None`
 - `• current_precept: 225c5ac9f5aa02673cf2dc5fe0834db7203be6aa`
 - `• sensor • changed_precept: 225c5ac9f5aa02673cf2dc5fe0834db7203be6aa`
 - `• sensor • string changed for Node: G, None: is now 225c5ac9f5aa02673cf2dc5fe0834db7203be6aa`
 - `• sensor • last_precept: None`
 - `• sensor • current_precept: 225c5ac9f5aa02673cf2dc5fe0834db7203be6aa`
 - `• get_impacted_nodes_actuator •these are the values of the tree and target_node {'A': {'B': {'C': {'D': {'E': {}}, 'F': {'G': {}, 'H': {}}, 'I': {'J': {}}}}}} and G`
 - `• get_impacted_nodes_actuator • subtree: {}`
 - `• get_all_descendants • Descendants: set()`
 - `• get_impacted_nodes_actuator • Impacted nodes: {'G'}`
 - `• sensor • Triggering pipelines for {'G'} to discover impact from changes detected in '225c5ac9f5aa02673cf2dc5fe0834db7203be6aa'`

 ```python
 """Goal Node: G
Nodes: A, B, C, D, E, F, G, H, I, J
Tree structure:
A
└── B
    └── C
        ├── D
        │   └── E
        ├── F
        │   ├── G  # Goal node
        │   └── H
        └── I
            └── J

DFS Traversal Path:
A -> B -> C -> D -> E -> F -> G (found)

Sensor Status:
- Current precept: <sha256_hash>
- Last precept: <different_sha256_hash>
- Change detected at Node G

Impacted Nodes:
- Target node: G
- Parent node: F
- Sibling node: H

Output: Triggering pipelines for {'F', 'G', 'H'}"""
```

 ```mermaid
graph TD;
    A --> B;
    B --> C;
    C --> D;
    D --> E;
    C --> F;
    F --> G;
    F --> H;
    C --> I;
    I --> J;

    %% Normal nodes (not impacted)
    style A fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style B fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style C fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style D fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style E fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style I fill:#3CB371,stroke:#2E8B57,stroke-width:2px;
    style J fill:#3CB371,stroke:#2E8B57,stroke-width:2px;

    %% Changed node (red)
    style G fill:#ff6b6b,stroke:#333,stroke-width:4px;
    
    %% Parent nodes affected by change (yellow)
    style F fill:#ffd93d,stroke:#333,stroke-width:3px;
    
    %% Sibling nodes affected (orange)
    style H fill:#ffa500,stroke:#333,stroke-width:2px;

```